In [26]:
import pandas as pd
import numpy as np
import joblib
import datetime

In [27]:
clf = joblib.load('catboost_multiclass_model.pkl')


In [28]:
scores=pd.read_csv('to_predict.csv')
scores = scores.drop(["goles_local", "goles_visitante", "fecha", "gana_local", 'empate', 'gana_visitante', 'arbitro'], axis=1)


In [29]:
scores_new = scores # [scores["fase"]=="todos contra todos"]
y_train = scores_new["result"]
x_train = scores_new.drop(["result"], axis=1)
x_train

,year,torneo,fase,jornada,local,visitante,dia,pos_local,pos_visitante,gol_favor_local_local_6_matches,...,los_visit_visit_3_matches,vict_local_igual_3_dire_matches,emp_local_igual_3_dire_matches,los_local_igual_3_dire_matches,vict_local_dif_3_dire_matches,emp_local_dif_3_dire_matches,los_local_dif_3_dire_matches,cuota_local,cuota_empate,cuota_visitante
0,2024,liga-dimayor-apertura,todos contra todos,10,Envigado FC,Jaguares De Córdoba,4,11,9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.05,3.40,3.90
1,2024,liga-dimayor-apertura,todos contra todos,10,Boyaca Chico FC,Tolima,4,18,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.50,3.20,3.10
2,2024,liga-dimayor-apertura,todos contra todos,10,Deportivo Cali,Once Caldas,4,5,11,1.833333,...,0.333333,0.0,1.0,0.0,0.333333,0.333333,0.333333,2.05,3.45,3.85
3,2024,liga-dimayor-apertura,todos contra todos,10,Millonarios,La Equidad,4,13,2,1.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.85,3.35,5.00
4,2024,liga-dimayor-apertura,todos contra todos,10,Deportivo Pereira,América de Cali,4,3,16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.45,3.30,3.10
5,2024,liga-dimayor-apertura,todos contra todos,10,CD Junior,Atlético Nacional,4,6,17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,3.45,4.00
6,2024,liga-dimayor-apertura,todos contra todos,10,Atletico Bucaramanga,Patriotas Boyaca,4,8,19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.90,3.45,4.75
7,2024,liga-dimayor-apertura,todos contra todos,10,Independiente Medellin,Rionegro Aguilas,4,15,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.25,3.35,3.35
8,2024,liga-dimayor-apertura,todos contra todos,10,Alianza FC,Independiente Santa Fe,4,14,4,1.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,3.50,2.05
9,2024,liga-dimayor-apertura,todos contra todos,10,Deportivo Pasto,Fortaleza FC,4,20,10,0.333333,...,0.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,2.30,3.20,3.40


In [30]:
soft_pred = clf.predict_proba(x_train)
hard_pred = clf.predict(x_train)
predictions = pd.concat([x_train, y_train], axis=1)

predictions["soft_pred_local"] = soft_pred[:,np.where(clf.classes_ == "Local")[0][0]]
predictions["soft_pred_empate"] = soft_pred[:,np.where(clf.classes_ == "Empate")[0][0]]
predictions["soft_pred_visitante"] = soft_pred[:,np.where(clf.classes_ == "Visitante")[0][0]]
predictions["hard_pred"] = hard_pred

predictions["expected_return_local"] = predictions["soft_pred_local"] * predictions["cuota_local"]
predictions["expected_return_empate"] = predictions["soft_pred_empate"] * predictions["cuota_empate"]
predictions["expected_return_visitante"] = predictions["soft_pred_visitante"] * predictions["cuota_visitante"]

predictions["best_bet"] = ""
predictions["expected_return"] = ""
for i in range(predictions.shape[0]):
    if (predictions.loc[i,"expected_return_local"] > predictions.loc[i, "expected_return_empate"]) & (predictions.loc[i,"expected_return_local"] > predictions.loc[i, "expected_return_visitante"]):
        predictions.loc[i, "best_bet"] = "local"
        predictions.loc[i, "expected_return"] = predictions.loc[i,"expected_return_local"]
    elif (predictions.loc[i,"expected_return_empate"] > predictions.loc[i, "expected_return_local"]) & (predictions.loc[i,"expected_return_empate"] > predictions.loc[i, "expected_return_visitante"]):
        predictions.loc[i, "best_bet"] = "empate"
        predictions.loc[i, "expected_return"] = predictions.loc[i,"expected_return_empate"]
    else:
        predictions.loc[i, "best_bet"] = "visitante"
        predictions.loc[i, "expected_return"] = predictions.loc[i,"expected_return_visitante"]
predictions = predictions.sort_values("expected_return", ascending=False)

predictions = predictions.drop(['gol_favor_local_local_6_matches',
       'gol_favor_local_visit_6_matches', 'gol_favor_visit_local_6_matches',
       'gol_favor_visit_visit_6_matches', 'gol_contra_local_local_6_matches',
       'gol_contra_local_visit_6_matches', 'gol_contra_visit_local_6_matches',
       'gol_contra_visit_visit_6_matches', 'vict_local_local_6_matches',
       'vict_local_visit_6_matches', 'vict_visit_local_6_matches',
       'vict_visit_visit_6_matches', 'emp_local_local_6_matches',
       'emp_local_visit_6_matches', 'emp_visit_local_6_matches',
       'emp_visit_visit_6_matches', 'los_local_local_6_matches',
       'los_local_visit_6_matches', 'los_visit_local_6_matches',
       'los_visit_visit_6_matches', 'vict_local_igual_6_dire_matches',
       'emp_local_igual_6_dire_matches', 'los_local_igual_6_dire_matches',
       'vict_local_dif_6_dire_matches', 'emp_local_dif_6_dire_matches',
       'los_local_dif_6_dire_matches', 'gol_favor_local_local_3_matches',
       'gol_favor_local_visit_3_matches', 'gol_favor_visit_local_3_matches',
       'gol_favor_visit_visit_3_matches', 'gol_contra_local_local_3_matches',
       'gol_contra_local_visit_3_matches', 'gol_contra_visit_local_3_matches',
       'gol_contra_visit_visit_3_matches', 'vict_local_local_3_matches',
       'vict_local_visit_3_matches', 'vict_visit_local_3_matches',
       'vict_visit_visit_3_matches', 'emp_local_local_3_matches',
       'emp_local_visit_3_matches', 'emp_visit_local_3_matches',
       'emp_visit_visit_3_matches', 'los_local_local_3_matches',
       'los_local_visit_3_matches', 'los_visit_local_3_matches',
       'los_visit_visit_3_matches', 'vict_local_igual_3_dire_matches',
       'emp_local_igual_3_dire_matches', 'los_local_igual_3_dire_matches',
       'vict_local_dif_3_dire_matches', 'emp_local_dif_3_dire_matches',
       'los_local_dif_3_dire_matches', 'result', 'dia'], axis=1)

current_timestamp = datetime.datetime.now()
predictions.to_csv("Predictions/predictions_catboost_multilabel" + str(current_timestamp).replace(':','_').replace('.','_').replace(' ','_').replace('-','_') + ".csv",index=False)
